# Performance i debugging

Techniki pomiaru wydajności, profilowania oraz narzędzia debuggera.


## Cele
- nauczyć się świadomie mierzyć wydajność (`timeit`, `perf_counter`)
- wykorzystać `cProfile` i `pstats` do analizy
- poznać podstawy debuggera (`pdb`) i narzędzi wizualnych


## Szybkie pomiary `timeit`
`timeit` uruchamia kod wielokrotnie, by uśrednić wyniki.


In [1]:
import timeit

# Tworzymy dane wejściowe tylko raz, poza pętlą pomiarową
setup = "nums = list(range(1000))"

# Mierzymy możliwie krótki fragment kodu, bez dodatkowych wywołań funkcji
stmt = "sum(nums)"

# Większa liczba powtórzeń zmniejsza wpływ szumu i planera
result = timeit.timeit(stmt=stmt, setup=setup, number=1000)
print(f'Średni czas: {result * 1e6/1000:.2f} µs')

Średni czas: 5.67 µs


## Profilowanie `cProfile`
`cProfile` to deterministyczny profiler napisany w C. Mierzy czas wykonania każdej funkcji i buduje drzewo wywołań, dzięki czemu szybko wyłapiesz sekcje kodu pochłaniające najwięcej czasu.

In [2]:
import cProfile
import pstats


def slow_function():
    total = 0
    for _ in range(10_000):
        subtotal = sum(range(100))  # sztuczne obciążenie CPU
        total += subtotal
    return total


with cProfile.Profile() as profiler:
    slow_function()  # kod, który chcemy przeanalizować

stats = pstats.Stats(profiler)
stats.strip_dirs()  # usuwa pełne ścieżki plików z raportu
stats.sort_stats(pstats.SortKey.CUMULATIVE).print_stats(8)

         10338 function calls (10334 primitive calls) in 0.035 seconds

   Ordered by: cumulative time
   List reduced from 100 to 8 due to restriction <8>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.034    0.034 events.py:92(_run)
        1    0.000    0.000    0.034    0.034 {method 'run' of '_contextvars.Context' objects}
        1    0.000    0.000    0.034    0.034 asyncio.py:206(_handle_events)
        1    0.000    0.000    0.034    0.034 zmqstream.py:573(_handle_events)
        1    0.000    0.000    0.034    0.034 zmqstream.py:614(_handle_recv)
        1    0.000    0.000    0.034    0.034 zmqstream.py:546(_run_callback)
    10000    0.024    0.000    0.024    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.021    0.021 threading.py:651(wait)




## Jak czytać raport `pstats`
Typowe kolumny w raporcie `print_stats` to:
- `ncalls` – liczba wywołań funkcji (druga liczba w nawiasie to tylko wywołania pierwotne).
- `tottime` – czas spędzony w funkcji, bez czasu funkcji wywoływanych z jej wnętrza.
- `percall` – `tottime / ncalls`, czyli średni koszt pojedynczego wywołania funkcji.
- `cumtime` – łączny czas funkcji wraz z funkcjami podrzędnymi.
- `percall` (druga kolumna) – `cumtime / primitive_calls`, przydatne przy rekurencji.
- `filename:lineno(function)` – miejsce w kodzie, które warto odwiedzić, aby zoptymalizować działanie.

`SortKey.CUMULATIVE` pomaga znaleźć funkcje, które spowalniają nasz program pośrednio, a `SortKey.TIME` skupia się na samym kodzie funkcji.

In [ ]:
# Sprawdźmy również, kto wywołuje najwolniejsze funkcje
stats.sort_stats(pstats.SortKey.TIME).print_callers(5)

## `cProfile` vs `profile` (czasem nazywanym `Profiler`)
- `cProfile` ma minimalny narzut (kod w C) i doskonale nadaje się do codziennej pracy.
- `profile` jest napisany w Pythonie, łatwo go rozszerzyć lub zmodyfikować, lecz spowalnia profilowany kod nawet kilkanaście razy.
- Oba moduły zwracają identyczne statystyki, więc możesz zaczynać od `cProfile`, a po prototypowaniu przełączyć się na `profile`, gdy potrzebujesz własnych haków.
- CLI: `python -m cProfile -o raport.prof skrypt.py` i później `snakeviz raport.prof` lub `python -m pstats raport.prof` do analizy offline.

In [ ]:
import profile

# `profile` działa identycznie jak `cProfile`, ale dodaje większy narzut czasowy
profile_profiler = profile.Profile()
profile_profiler.runcall(slow_function)

stats_py = pstats.Stats(profile_profiler)
stats_py.strip_dirs()
stats_py.sort_stats(pstats.SortKey.CUMULATIVE).print_stats(5)

## Debugowanie z `pdb`
Debugger pozwala zatrzymać program, obejrzeć stan i kontynuować.


In [ ]:
from bdb import BdbQuit


def buggy(numbers):
    total = 0
    for value in numbers:
        if value % 2 == 0:
            total += value
        else:
            import pdb; pdb.set_trace()  # pułapka na nieparzyste wartości
    return total


try:
    buggy([2, 4, 5])  # debugger zatrzyma się na pierwszej liczbie nieparzystej
except BdbQuit:
    pass  # użytkownik zakończył sesję debuggera

    [... skipping 21 hidden frame(s)]
  /var/folders/pd/gsgbf0fd4tv7ntlsxt53r0sh0000gn/T/ipykernel_34263/3958656088.py(11)<module>()
      9 
     10 try:
---> 11     buggy([2, 4, 5])
     12 except BdbQuit:
     13     pass
> /var/folders/pd/gsgbf0fd4tv7ntlsxt53r0sh0000gn/T/ipykernel_34263/3958656088.py(7)buggy()
      5             total += value
      6         else:
----> 7             import pdb; pdb.set_trace()  # pułapka na nieparzyste wartości
      8     return total
      9 



ipdb>  


**Podsumowanie:** Regularne pomiary i profilowanie pomagają zlokalizować wąskie gardła.

**Pytanie kontrolne:** Dlaczego `print` nie wystarcza do debugowania złożonych aplikacji?


### 🧩 Zadanie 1
Napisz funkcję, która porównuje wydajność dwóch implementacji algorytmu i raportuje różnicę procentową.


In [ ]:
# Rozwiązanie Zadania 1
import timeit


def benchmark(func_a, func_b, number=1000):
    # Mierzymy obie funkcje w izolacji, aby wynik nie zależał od siebie nawzajem
    time_a = timeit.timeit(func_a, number=number)
    time_b = timeit.timeit(func_b, number=number)
    diff = ((time_b - time_a) / time_a) * 100
    print(f'A: {time_a:.6f}s, B: {time_b:.6f}s, różnica: {diff:.2f}%')


benchmark(lambda: sum(range(1000)), lambda: sum([*range(1000)]))

### 🧩 Zadanie 2
Stwórz funkcję, która profiluje dowolną funkcję i zwraca pięć najbardziej czasochłonnych wpisów.


In [1]:
# Rozwiązanie Zadania 2
import cProfile
import pstats
from io import StringIO


def profile_top(func, *args, **kwargs):
    with cProfile.Profile() as pr:
        func(*args, **kwargs)
    s = StringIO()
    stats = pstats.Stats(pr, stream=s)
    stats.sort_stats(pstats.SortKey.CUMULATIVE).print_stats(5)
    print(s.getvalue())  # raport jako tekst, gotowy do logów lub e-maili


def busy_work():
    total = 0
    for _ in range(1000):
        total += sum(range(500))  # intensywna pętla do profilowania
    return total


profile_top(busy_work)

         1109 function calls in 0.015 seconds

   Ordered by: cumulative time
   List reduced from 46 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.014    0.000    0.014    0.000 {built-in method builtins.sum}
        1    0.000    0.000    0.011    0.011 /Users/rkorzen/.local/share/uv/python/cpython-3.14.0-macos-x86_64-none/lib/python3.14/threading.py:651(wait)
        1    0.000    0.000    0.011    0.011 /Users/rkorzen/.local/share/uv/python/cpython-3.14.0-macos-x86_64-none/lib/python3.14/threading.py:337(wait)
        1    0.001    0.001    0.011    0.011 /var/folders/pd/gsgbf0fd4tv7ntlsxt53r0sh0000gn/T/ipykernel_7762/4138807054.py:16(busy_work)
        2    0.000    0.000    0.000    0.000 /Users/rkorzen/workspace/.venv/lib/python3.14/site-packages/traitlets/traitlets.py:708(__set__)



